In [16]:
import cv2
import os
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import csv
from datetime import datetime

# Paths
MODEL_PATH = r"D:\PROJECT\model\cnn_face_recog.keras"
DATASET_PATH = r"D:\PROJECT\dataset"
PROTOTXT_PATH = r"D:\PROJECT\deploy.prototxt"
WEIGHTS_PATH = r"D:\PROJECT\res10_300x300_ssd_iter_140000.caffemodel"
ATTENDANCE_FILE = r"D:\PROJECT\attendance.csv"

# Load OpenCV DNN face detector
net = cv2.dnn.readNetFromCaffe(PROTOTXT_PATH, WEIGHTS_PATH)

# Load trained CNN model
model = load_model(MODEL_PATH)

# Class labels (folder names under dataset/)
class_labels = sorted(os.listdir(DATASET_PATH))

# Attendance marking function
def mark_attendance(name):
    now = datetime.now()
    time_str = now.strftime("%H:%M:%S")
    date_str = now.strftime("%Y-%m-%d")
    
    if not os.path.isfile(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Name", "Date", "Time"])

    with open(ATTENDANCE_FILE, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([name, date_str, time_str])

# Register student function
def register_student():
    name = simpledialog.askstring("Register Student", "Enter student name:")
    if not name:
        return

    path = os.path.join(DATASET_PATH, name)
    os.makedirs(path, exist_ok=True)
    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                                     (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = frame[y1:y2, x1:x2]
                if face.size == 0:
                    continue
                face = cv2.resize(face, (100, 100))
                cv2.imwrite(os.path.join(path, f"{count}.jpg"), face)
                count += 1
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.imshow("Register - Press Q to exit", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or count >= 50:
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Success", f"Captured {count} images for {name}.")

# Start attendance function
def start_attendance():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                                     (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = frame[y1:y2, x1:x2]
                if face.size == 0:
                    continue

                # Convert face to grayscale
                face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                face_gray = cv2.resize(face_gray, (100, 100))  # Resize for CNN model
                face_gray = face_gray.astype("float32") / 255.0
                face_gray = np.expand_dims(face_gray, axis=-1)  # Add channel dimension

                # Predict the identity using the CNN model
                face_input = np.expand_dims(face_gray, axis=0)  # Add batch dimension
                pred = model.predict(face_input, verbose=0)[0]
                label_idx = np.argmax(pred)
                name = class_labels[label_idx]

                mark_attendance(name)
                label = f"{name}: {pred[label_idx]*100:.2f}%"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Attendance - Press Q to exit", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# GUI setup
root = tk.Tk()
root.title("Face Recognition Attendance System")
root.geometry("400x200")

register_btn = tk.Button(root, text="Register Student", command=register_student, font=('Arial', 14))
register_btn.pack(pady=20)

attendance_btn = tk.Button(root, text="Start Attendance", command=start_attendance, font=('Arial', 14))
attendance_btn.pack(pady=20)

root.mainloop()

